# LLM Model Parallelism: From 1D to 5D Parallelism
## Comprehensive Analysis with Qwen 2.5 3B

This notebook demonstrates progressive model parallelism strategies for large language models, testing from 2 GPUs to 8 GPUs with comprehensive metrics tracking.

**Model**: Qwen 2.5 3B (~3B parameters)  
**Dataset**: MetaMathQA (395K math QA examples) - Industry-standard math reasoning dataset  
**Frameworks**: DeepSpeed, PyTorch Native  
**Progression**: DDP → 2D → 3D → 4D → 5D Parallelism


In [ ]:
import torch
import sys
import os
import time
import warnings
from typing import Dict, List, Optional
import json

print("=== Environment Setup ===")
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")

warnings.filterwarnings("ignore")
print("\n Environment check completed!")


In [ ]:
!nvidia-smi


## Install Dependencies


In [ ]:
%pip install -q transformers datasets accelerate deepspeed wandb sentencepiece protobuf


## Setup WandB


In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

# Get the secret value
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# Log in using the key
wandb.login(key=wandb_api_key)

print(" Successfully logged into W&B!")


## Utility Functions: Metrics and MFU Calculation


In [ ]:
"""
Comprehensive metrics tracking and MFU calculation for LLM training.
"""

def calculate_llama_flops_per_token(config, seq_len=512):
    """
    Calculate FLOPs for transformer forward+backward pass.
    Based on Qwen/LLaMA architecture.
    """
    num_layers = config.num_hidden_layers
    hidden_size = config.hidden_size
    intermediate_size = config.intermediate_size
    num_heads = config.num_attention_heads
    vocab_size = config.vocab_size
    
    # Embedding FLOPs (approximate)
    embedding_flops = seq_len * hidden_size * 2
    
    # Per-layer FLOPs
    # Attention: QKV projection + attention computation + output projection
    attention_flops = (
        3 * seq_len * hidden_size * hidden_size +  # QKV projection
        2 * seq_len * seq_len * hidden_size +      # Attention computation
        seq_len * hidden_size * hidden_size        # Output projection
    )
    
    # MLP (Gated MLP: gate + up + down for Qwen)
    mlp_flops = (
        2 * seq_len * hidden_size * intermediate_size +  # Gate + Up
        seq_len * intermediate_size * hidden_size          # Down
    )
    
    # LayerNorm (2 per layer)
    layernorm_flops = 4 * seq_len * hidden_size
    
    layer_flops = attention_flops + mlp_flops + layernorm_flops
    
    # Total forward FLOPs
    forward_flops = embedding_flops + num_layers * layer_flops
    
    # LM Head
    lm_head_flops = seq_len * hidden_size * vocab_size
    
    # Forward + Backward (backward ~2x forward)
    total_flops = (forward_flops + lm_head_flops) * 3
    
    return total_flops

def get_theoretical_flops_per_second(num_gpus=2, gpu_model="T4"):
    """Get theoretical peak FLOPs per second."""
    gpu_tflops = {
        "T4": 65,
        "V100": 125,
        "A100": 312,
        "A6000": 150,
        "RTX3090": 142,
        "RTX4090": 330
    }
    
    single_gpu_tflops = gpu_tflops.get(gpu_model, 65)
    total_tflops = single_gpu_tflops * num_gpus
    return total_tflops * 1e12  # Convert to FLOPS

def calculate_mfu(model, config, batch_size, seq_len, step_time, num_gpus=2, gpu_model="T4"):
    """Calculate Model FLOPs Utilization (MFU)."""
    flops_per_token = calculate_llama_flops_per_token(config, seq_len)
    total_flops = flops_per_token * batch_size * seq_len
    
    actual_flops_per_sec = total_flops / step_time
    theoretical_flops_per_sec = get_theoretical_flops_per_second(num_gpus, gpu_model)
    
    mfu = (actual_flops_per_sec / theoretical_flops_per_sec) * 100
    
    return mfu, actual_flops_per_sec, theoretical_flops_per_sec

class MetricsTracker:
    """Comprehensive metrics tracking for distributed training."""
    
    def __init__(self, rank=0):
        self.rank = rank
        self.metrics = {
            "step_times": [],
            "forward_times": [],
            "backward_times": [],
            "optimizer_times": [],
            "communication_times": [],
            "memory_allocated": [],
            "memory_reserved": [],
            "memory_peak": [],
            "losses": [],
            "throughput_samples": [],
            "throughput_tokens": [],
        }
    
    def record_step(self, step_metrics: Dict):
        """Record metrics for a single step."""
        if self.rank == 0:
            for key, value in step_metrics.items():
                if key in self.metrics:
                    self.metrics[key].append(value)
    
    def get_summary(self) -> Dict:
        """Get summary statistics."""
        if self.rank != 0:
            return {}
        
        summary = {}
        for key, values in self.metrics.items():
            if values:
                summary[f"{key}_mean"] = sum(values) / len(values)
                summary[f"{key}_min"] = min(values)
                summary[f"{key}_max"] = max(values)
        
        return summary

print(" Metrics utilities loaded!")


In [ ]:
# Kaggle Setup: Copy metamathqa_utils.py from input dataset
import shutil
import os
import sys

# Source path in Kaggle input dataset
kaggle_input_path = "/kaggle/input/mathqa-utils/metamathqa_utils.py"
working_dir_path = "./metamathqa_utils.py"

# Copy file from Kaggle input to working directory
if os.path.exists(kaggle_input_path):
    shutil.copy(kaggle_input_path, working_dir_path)
    print(f"Copied metamathqa_utils.py from Kaggle input to working directory")
elif os.path.exists(working_dir_path):
    print("Using existing metamathqa_utils.py in working directory")
else:
    # Fallback: add Kaggle input path to sys.path
    kaggle_utils_dir = "/kaggle/input/mathqa-utils"
    if os.path.exists(kaggle_utils_dir):
        sys.path.insert(0, kaggle_utils_dir)
        print(f"Added {kaggle_utils_dir} to Python path")
    else:
        print("ERROR: metamathqa_utils.py not found. Please check dataset attachment.")

# Test the utility module
from metamathqa_utils import load_metamathqa_dataset

# Load dataset (small subset for testing)
print("Testing MetaMathQA dataset loading...")
tokenized_dataset, tokenizer = load_metamathqa_dataset(split="train[:100]", rank=0)

print(f"\nDataset loaded successfully!")
print(f"Number of examples: {len(tokenized_dataset)}")
print(f"Sample keys: {tokenized_dataset[0].keys()}")
print(f"Sequence length: {len(tokenized_dataset[0]['input_ids'])}")
print(f"\nDataset Configuration:")
print(f"  - Model: Qwen 2.5 3B-Instruct")
print(f"  - Dataset: MetaMathQA (meta-math/MetaMathQA)")
print(f"  - Format: Instruction-following (Qwen chat template)")
print(f"  - Max sequence length: 2048 tokens")
print(f"  - Full dataset size: ~395K examples")

# Part 1: 1D Parallelism - Data Parallelism (DDP)
## Baseline: Standard DistributedDataParallel


In [ ]:
%%writefile train_ddp_qwen3b_metamath.py
"""
1D Parallelism: Data Parallelism (DDP)
Baseline experiment with standard PyTorch DDP using MetaMathQA dataset.
"""

import torch
import torch.nn as nn
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
from torch.cuda.amp import autocast, GradScaler
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_from_disk
import warnings
import wandb
import os
import time
import json
import torch.profiler

warnings.filterwarnings("ignore")

# MFU calculation functions
def calculate_llama_flops_per_token(config, seq_len=512):
    """Calculate FLOPs for transformer forward+backward pass."""
    num_layers = config.num_hidden_layers
    hidden_size = config.hidden_size
    intermediate_size = config.intermediate_size
    vocab_size = config.vocab_size
    
    embedding_flops = seq_len * hidden_size * 2
    attention_flops = (
        3 * seq_len * hidden_size * hidden_size +
        2 * seq_len * seq_len * hidden_size +
        seq_len * hidden_size * hidden_size
    )
    mlp_flops = (
        2 * seq_len * hidden_size * intermediate_size +
        seq_len * intermediate_size * hidden_size
    )
    layernorm_flops = 4 * seq_len * hidden_size
    layer_flops = attention_flops + mlp_flops + layernorm_flops
    forward_flops = embedding_flops + num_layers * layer_flops
    lm_head_flops = seq_len * hidden_size * vocab_size
    total_flops = (forward_flops + lm_head_flops) * 3
    return total_flops

def get_theoretical_flops_per_second(num_gpus=2, gpu_model="T4"):
    """Get theoretical peak FLOPs per second."""
    gpu_tflops = {"T4": 65, "V100": 125, "A100": 312, "A6000": 150, "RTX3090": 142, "RTX4090": 330}
    single_gpu_tflops = gpu_tflops.get(gpu_model, 65)
    return single_gpu_tflops * num_gpus * 1e12

def calculate_mfu(model, config, batch_size, seq_len, step_time, num_gpus, gpu_model):
    """Calculate Model FLOPs Utilization (MFU)."""
    flops_per_token = calculate_llama_flops_per_token(config, seq_len)
    total_flops = flops_per_token * batch_size * seq_len
    actual_flops_per_sec = total_flops / step_time
    theoretical_flops_per_sec = get_theoretical_flops_per_second(num_gpus, gpu_model)
    mfu = (actual_flops_per_sec / theoretical_flops_per_sec) * 100
    return mfu, actual_flops_per_sec, theoretical_flops_per_sec

# Setup
local_rank = int(os.environ['LOCAL_RANK'])
rank = int(os.environ['RANK'])
world_size = int(os.environ['WORLD_SIZE'])

torch.cuda.set_device(local_rank)
device = torch.device(f"cuda:{local_rank}")

dist.init_process_group(backend='nccl')

# WandB
if rank == 0:
    wandb.init(
        project="LLM-Model-Parallelism-Qwen3B",
        name=f"DDP-Qwen3B-MetaMathQA-{world_size}GPUs",
        config={
            "model": "Qwen2.5-3B",
            "dataset": "MetaMathQA",
            "parallelism": "1D-Data-Parallelism",
            "framework": "PyTorch DDP",
            "num_gpus": world_size
        }
    )

# Load model
if rank == 0:
    print("Loading Qwen 2.5 3B model...")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    torch_dtype=torch.float16,
    device_map=None
)

model = model.to(device)
model = DDP(model, device_ids=[local_rank], find_unused_parameters=False)

# Data setup - MetaMathQA using shared utility
from metamathqa_utils import load_metamathqa_dataset

tokenized_dataset, tokenizer = load_metamathqa_dataset(
    split="train[:1000]",  # Use subset for testing
    cache_dir="./metamathqa_tokenized_data",
    rank=rank
)

if rank == 0:
    print(f"MetaMathQA dataset loaded: {len(tokenized_dataset)} examples")

# DataLoader
from torch.utils.data import DataLoader

MICRO_BATCH_SIZE = 1  # Small batch for 3B model
train_sampler = DistributedSampler(tokenized_dataset, num_replicas=world_size, rank=rank, shuffle=True)
train_loader = DataLoader(
    tokenized_dataset,
    batch_size=MICRO_BATCH_SIZE,
    sampler=train_sampler,
    collate_fn=lambda x: {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in x]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in x]),
        'labels': torch.stack([torch.tensor(item['labels']) for item in x])
    }
)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scaler = GradScaler()

# Profiler trace handler
def trace_handler(prof):
    if rank == 0:
        trace_dir = "./profiler_logs/llm_1d_ddp_trace"
        os.makedirs(trace_dir, exist_ok=True)
        prof.export_chrome_trace(f"{trace_dir}/rank{rank}_trace.json")
        print(f" Profiler trace saved to {trace_dir}/rank{rank}_trace.json")

# Training loop
model.train()
config = model.module.config if hasattr(model, 'module') else model.config

# Profiler schedule: wait=1, warmup=1, active=3 (capture 3 steps), repeat=1
with torch.profiler.profile(
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
    on_trace_ready=trace_handler,
    record_shapes=True,
    with_stack=True,
    profile_memory=True
) as prof:
    
    for epoch in range(1):
        train_sampler.set_epoch(epoch)
        
        for i, batch in enumerate(train_loader):
            if i >= 20:  # Short run for testing
                break
            
            step_start_time = time.time()
            
            # Label training step
            with torch.profiler.record_function(f"## Training Step {i} ##"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                # Mask padding tokens in labels
                labels[labels == tokenizer.pad_token_id] = -100
                
                optimizer.zero_grad()
                
                # Label forward pass
                forward_start = time.time()
                with torch.profiler.record_function("## Forward Pass ##"):
                    with autocast():
                        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                        loss = outputs.loss
                forward_time = time.time() - forward_start
                
                # Label backward pass
                backward_start = time.time()
                with torch.profiler.record_function("## Backward Pass ##"):
                    scaler.scale(loss).backward()
                backward_time = time.time() - backward_start
                
                # Label optimizer step
                optimizer_start = time.time()
                with torch.profiler.record_function("## Optimizer Step ##"):
                    scaler.step(optimizer)
                    scaler.update()
                optimizer_time = time.time() - optimizer_start
            
            step_time = time.time() - step_start_time
            
            # Step profiler (only for first few steps)
            if i < 5:  # Profile first 5 steps (wait=1, warmup=1, active=3)
                prof.step()
        
        # Calculate MFU
        effective_batch = MICRO_BATCH_SIZE * world_size
        seq_len = input_ids.size(1)
        mfu, actual_flops, theoretical_flops = calculate_mfu(
            model.module if hasattr(model, 'module') else model,
            config,
            effective_batch,
            seq_len,
            step_time,
            num_gpus=world_size,
            gpu_model="T4"
        )
        
        if rank == 0:
            mem_allocated = torch.cuda.memory_allocated() / 1e9
            mem_reserved = torch.cuda.memory_reserved() / 1e9
            throughput_samples = effective_batch / step_time
            throughput_tokens = effective_batch * seq_len / step_time
            
            wandb.log({
                "loss": loss.item(),
                "mfu_percent": mfu,
                "step_time_sec": step_time,
                "forward_time_sec": forward_time,
                "backward_time_sec": backward_time,
                "optimizer_time_sec": optimizer_time,
                "memory_allocated_gb": mem_allocated,
                "memory_reserved_gb": mem_reserved,
                "throughput_samples_per_sec": throughput_samples,
                "throughput_tokens_per_sec": throughput_tokens,
                "actual_tflops": actual_flops / 1e12,
                "theoretical_tflops": theoretical_flops / 1e12
            })
            
            print(f"Step {i}: Loss={loss.item():.4f} | MFU={mfu:.2f}% | Time={step_time:.2f}s | Mem={mem_allocated:.2f}GB")

if rank == 0:
    wandb.finish()
dist.destroy_process_group()


In [ ]:
# Run DDP experiment on 2 GPUs with MetaMathQA
# !torchrun --nproc_per_node=2 train_ddp_qwen3b_metamath.py


# Part 2: 2D Parallelism - Data + Pipeline Parallelism
## Combining Data Parallelism with Pipeline Parallelism


In [ ]:
%%writefile train_2d_pipeline_qwen3b_metamath.py
"""
2D Parallelism: Data Parallelism + Pipeline Parallelism
Using DeepSpeed Pipeline Parallelism with MetaMathQA dataset.
"""

import torch
import torch.nn as nn
import deepspeed
import torch.distributed as dist
from transformers import AutoModelForCausalLM
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader
from metamathqa_utils import load_metamathqa_dataset
import warnings
import wandb
import os
import time
import torch.profiler

warnings.filterwarnings("ignore")

# Setup
local_rank = int(os.environ['LOCAL_RANK'])
rank = int(os.environ['RANK'])
world_size = int(os.environ['WORLD_SIZE'])

torch.cuda.set_device(local_rank)
device = torch.device(f"cuda:{local_rank}")

# DeepSpeed config for 2D parallelism (Pipeline + Data)
deepspeed_config = {
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": 1,
    "gradient_accumulation_steps": 1,
    "fp16": {"enabled": True},
    "zero_optimization": {"stage": 0},  # No ZeRO for pure pipeline parallelism
    "pipeline": {
        "stages": world_size,  # Each GPU is a pipeline stage
        "partition": "type"
    },
    "wall_clock_breakdown": True
}

# Initialize DeepSpeed
deepspeed.init_distributed()

if rank == 0:
    wandb.init(
        project="LLM-Model-Parallelism-Qwen3B",
        name=f"2D-Pipeline-Data-Qwen3B-MetaMathQA-{world_size}GPUs",
        config={
            "model": "Qwen2.5-3B",
            "dataset": "MetaMathQA",
            "parallelism": "2D-Pipeline-Data",
            "framework": "DeepSpeed",
            "num_gpus": world_size,
            "pipeline_stages": world_size
        }
    )

# Load model
if rank == 0:
    print("Loading Qwen 2.5 3B model...")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    torch_dtype=torch.float16,
    device_map=None
)

# Initialize DeepSpeed with pipeline parallelism
model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    model_parameters=model.parameters(),
    config=deepspeed_config
)

# Data setup - MetaMathQA using shared utility
tokenized_dataset, tokenizer = load_metamathqa_dataset(
    split="train[:1000]",
    cache_dir="./metamathqa_tokenized_data",
    rank=rank
)

if rank == 0:
    print(f"MetaMathQA dataset loaded: {len(tokenized_dataset)} examples")

MICRO_BATCH_SIZE = 1
train_sampler = DistributedSampler(tokenized_dataset, num_replicas=world_size, rank=rank, shuffle=True)
train_loader = DataLoader(
    tokenized_dataset,
    batch_size=MICRO_BATCH_SIZE,
    sampler=train_sampler,
    collate_fn=lambda x: {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in x]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in x]),
        'labels': torch.stack([torch.tensor(item['labels']) for item in x])
    }
)

# Profiler trace handler
def trace_handler(prof):
    if rank == 0:
        trace_dir = "./profiler_logs/llm_2d_pipeline_trace"
        os.makedirs(trace_dir, exist_ok=True)
        prof.export_chrome_trace(f"{trace_dir}/rank{rank}_trace.json")
        print(f"Profiler trace saved to {trace_dir}/rank{rank}_trace.json")

# Training loop
model_engine.train()
config = model_engine.module.config if hasattr(model_engine, 'module') else model.config

# Profiler schedule: wait=1, warmup=1, active=3 (capture 3 steps), repeat=1
with torch.profiler.profile(
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
    on_trace_ready=trace_handler,
    record_shapes=True,
    with_stack=True,
    profile_memory=True
) as prof:
    
    for epoch in range(1):
        train_sampler.set_epoch(epoch)
        
        for i, batch in enumerate(train_loader):
            if i >= 20:
                break
            
            step_start_time = time.time()
            
            # Label training step
            with torch.profiler.record_function(f"## Training Step {i} ##"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                # Mask padding tokens in labels
                labels[labels == tokenizer.pad_token_id] = -100
                
                # Label forward pass
                forward_start = time.time()
                with torch.profiler.record_function("## Forward Pass ##"):
                    outputs = model_engine(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                forward_time = time.time() - forward_start
                
                # Label backward pass
                backward_start = time.time()
                with torch.profiler.record_function("## Backward Pass ##"):
                    model_engine.backward(loss)
                backward_time = time.time() - backward_start
                
                # Label optimizer step
                optimizer_start = time.time()
                with torch.profiler.record_function("## Optimizer Step ##"):
                    model_engine.step()
                optimizer_time = time.time() - optimizer_start
            
            step_time = time.time() - step_start_time
            
            # Step profiler (only for first few steps)
            if i < 5:  # Profile first 5 steps (wait=1, warmup=1, active=3)
                prof.step()
        
        if rank == 0:
            mem_allocated = torch.cuda.memory_allocated() / 1e9
            effective_batch = MICRO_BATCH_SIZE * world_size
            seq_len = input_ids.size(1)
            throughput_samples = effective_batch / step_time
            throughput_tokens = effective_batch * seq_len / step_time
            
            wandb.log({
                "loss": loss.item(),
                "step_time_sec": step_time,
                "forward_time_sec": forward_time,
                "backward_time_sec": backward_time,
                "optimizer_time_sec": optimizer_time,
                "memory_allocated_gb": mem_allocated,
                "throughput_samples_per_sec": throughput_samples,
                "throughput_tokens_per_sec": throughput_tokens
            })
            
            print(f"Step {i}: Loss={loss.item():.4f} | Time={step_time:.2f}s | Mem={mem_allocated:.2f}GB")

if rank == 0:
    wandb.finish()
dist.destroy_process_group()


In [ ]:
# Run 2D Pipeline experiment on 2 GPUs
# !deepspeed --num_gpus=2 train_2d_pipeline_qwen3b_metamath.py


# Part 3: 3D Parallelism - Data + Pipeline + Tensor Parallelism
## Full 3D Parallelism: The Standard for Large-Scale Training


In [ ]:
%%writefile train_3d_parallelism_qwen3b_metamath.py
"""
3D Parallelism: Data + Pipeline + Tensor Parallelism
Combining all three parallelism strategies with MetaMathQA dataset.
For 2 GPUs: 2 pipeline stages, 1-way tensor parallelism
For 8 GPUs: 2 pipeline stages × 2-way tensor × 2 data parallel
"""

import torch
import deepspeed
import torch.distributed as dist
from transformers import AutoModelForCausalLM
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader
from metamathqa_utils import load_metamathqa_dataset
import warnings
import wandb
import os
import time
import torch.profiler

warnings.filterwarnings("ignore")

# Setup
local_rank = int(os.environ['LOCAL_RANK'])
rank = int(os.environ['RANK'])
world_size = int(os.environ['WORLD_SIZE'])

torch.cuda.set_device(local_rank)
device = torch.device(f"cuda:{local_rank}")

# 3D Parallelism Configuration
if world_size == 2:
    pp_size = 2
    tp_size = 1
    dp_size = 1
elif world_size == 8:
    pp_size = 2
    tp_size = 2
    dp_size = 2
else:
    pp_size = 2
    tp_size = world_size // 4
    dp_size = world_size // (pp_size * tp_size)

deepspeed_config = {
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": 1,
    "gradient_accumulation_steps": 1,
    "fp16": {"enabled": True},
    "zero_optimization": {"stage": 0},
    "pipeline": {"stages": pp_size, "partition": "type"},
    "tensor_parallel": {"tp_size": tp_size},
    "wall_clock_breakdown": True
}

deepspeed.init_distributed()

if rank == 0:
    wandb.init(
        project="LLM-Model-Parallelism-Qwen3B",
        name=f"3D-Parallelism-Qwen3B-MetaMathQA-{world_size}GPUs",
        config={
            "model": "Qwen2.5-3B",
            "dataset": "MetaMathQA",
            "parallelism": "3D-Data-Pipeline-Tensor",
            "framework": "DeepSpeed",
            "num_gpus": world_size,
            "pipeline_stages": pp_size,
            "tensor_parallel_size": tp_size,
            "data_parallel_size": dp_size
        }
    )

# Load model
if rank == 0:
    print(f"Loading Qwen 2.5 3B with 3D parallelism: PP={pp_size}, TP={tp_size}, DP={dp_size}")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    torch_dtype=torch.float16,
    device_map=None
)

model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    model_parameters=model.parameters(),
    config=deepspeed_config
)

# Data setup - MetaMathQA using shared utility
tokenized_dataset, tokenizer = load_metamathqa_dataset(
    split="train[:1000]",
    cache_dir="./metamathqa_tokenized_data",
    rank=rank
)

if rank == 0:
    print(f"MetaMathQA dataset loaded: {len(tokenized_dataset)} examples")

MICRO_BATCH_SIZE = 1
train_sampler = DistributedSampler(tokenized_dataset, num_replicas=world_size, rank=rank, shuffle=True)
train_loader = DataLoader(
    tokenized_dataset,
    batch_size=MICRO_BATCH_SIZE,
    sampler=train_sampler,
    collate_fn=lambda x: {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in x]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in x]),
        'labels': torch.stack([torch.tensor(item['labels']) for item in x])
    }
)

# Profiler trace handler
def trace_handler(prof):
    if rank == 0:
        trace_dir = "./profiler_logs/llm_3d_parallelism_trace"
        os.makedirs(trace_dir, exist_ok=True)
        prof.export_chrome_trace(f"{trace_dir}/rank{rank}_trace.json")
        print(f"Profiler trace saved to {trace_dir}/rank{rank}_trace.json")

# Training loop
model_engine.train()

# Profiler schedule: wait=1, warmup=1, active=3 (capture 3 steps), repeat=1
with torch.profiler.profile(
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
    on_trace_ready=trace_handler,
    record_shapes=True,
    with_stack=True,
    profile_memory=True
) as prof:
    
    for epoch in range(1):
        train_sampler.set_epoch(epoch)
        
        for i, batch in enumerate(train_loader):
            if i >= 20:
                break
            
            step_start_time = time.time()
            
            # Label training step
            with torch.profiler.record_function(f"## Training Step {i} ##"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                labels[labels == tokenizer.pad_token_id] = -100
                
                # Label forward pass
                with torch.profiler.record_function("## Forward Pass ##"):
                    outputs = model_engine(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                
                # Label backward pass
                with torch.profiler.record_function("## Backward Pass ##"):
                    model_engine.backward(loss)
                
                # Label optimizer step
                with torch.profiler.record_function("## Optimizer Step ##"):
                    model_engine.step()
            
            step_time = time.time() - step_start_time
            
            # Step profiler (only for first few steps)
            if i < 5:  # Profile first 5 steps (wait=1, warmup=1, active=3)
                prof.step()
        
        if rank == 0:
            mem_allocated = torch.cuda.memory_allocated() / 1e9
            effective_batch = MICRO_BATCH_SIZE * dp_size
            seq_len = input_ids.size(1)
            throughput_samples = effective_batch / step_time
            throughput_tokens = effective_batch * seq_len / step_time
            
            wandb.log({
                "loss": loss.item(),
                "step_time_sec": step_time,
                "memory_allocated_gb": mem_allocated,
                "throughput_samples_per_sec": throughput_samples,
                "throughput_tokens_per_sec": throughput_tokens
            })
            
            print(f"Step {i}: Loss={loss.item():.4f} | Time={step_time:.2f}s | Mem={mem_allocated:.2f}GB")

if rank == 0:
    wandb.finish()
dist.destroy_process_group()


In [ ]:
# Run 3D Parallelism experiment on 2 GPUs
# !deepspeed --num_gpus=2 train_3d_parallelism_qwen3b_metamath.py


# Part 4: 4D Parallelism - Adding Sequence Parallelism
## Data + Pipeline + Tensor + Sequence Parallelism


In [ ]:
%%writefile train_4d_parallelism_qwen3b_metamath.py
"""
4D Parallelism: Data + Pipeline + Tensor + Sequence Parallelism
Sequence parallelism splits the sequence dimension across GPUs.
This is particularly useful for long sequences in MetaMathQA.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import deepspeed
import torch.distributed as dist
from transformers import AutoModelForCausalLM
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader
from metamathqa_utils import load_metamathqa_dataset
import warnings
import wandb
import os
import time
import torch.profiler

warnings.filterwarnings("ignore")

# Setup
local_rank = int(os.environ['LOCAL_RANK'])
rank = int(os.environ['RANK'])
world_size = int(os.environ['WORLD_SIZE'])

torch.cuda.set_device(local_rank)
device = torch.device(f"cuda:{local_rank}")

# 4D Parallelism Configuration
if world_size == 2:
    dp_size = 1
    pp_size = 2
    tp_size = 1
    sp_size = 1
elif world_size == 8:
    dp_size = 2
    pp_size = 2
    tp_size = 1
    sp_size = 2
else:
    dp_size = 1
    pp_size = 2
    tp_size = 1
    sp_size = world_size // (dp_size * pp_size * tp_size)

deepspeed.init_distributed()

if rank == 0:
    wandb.init(
        project="LLM-Model-Parallelism-Qwen3B",
        name=f"4D-Parallelism-Qwen3B-MetaMathQA-{world_size}GPUs",
        config={
            "model": "Qwen2.5-3B",
            "dataset": "MetaMathQA",
            "parallelism": "4D-Data-Pipeline-Tensor-Sequence",
            "framework": "DeepSpeed + Custom SP",
            "num_gpus": world_size,
            "data_parallel_size": dp_size,
            "pipeline_stages": pp_size,
            "tensor_parallel_size": tp_size,
            "sequence_parallel_size": sp_size
        }
    )

# Custom Sequence Parallel Attention Layer (conceptual)
class SequenceParallelAttention(nn.Module):
    """Attention layer with sequence parallelism."""
    def __init__(self, config, rank, world_size, sp_size):
        super().__init__()
        self.rank = rank
        self.sp_size = sp_size
        self.num_heads = config.num_attention_heads
        self.head_dim = config.hidden_size // config.num_attention_heads
        self.hidden_size = config.hidden_size
        self.qkv = nn.Linear(config.hidden_size, 3 * config.hidden_size, bias=False)
        self.out_proj = nn.Linear(config.hidden_size, config.hidden_size)
        
    def forward(self, hidden_states):
        batch_size, seq_len, hidden_size = hidden_states.shape
        seq_len_per_rank = seq_len // self.sp_size
        start_idx = (self.rank % self.sp_size) * seq_len_per_rank
        end_idx = start_idx + seq_len_per_rank
        
        local_hidden = hidden_states[:, start_idx:end_idx, :]
        qkv = self.qkv(local_hidden)
        qkv = qkv.reshape(batch_size, seq_len_per_rank, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attn_weights = F.softmax(scores, dim=-1)
        attn_output = torch.matmul(attn_weights, v)
        
        attn_output = attn_output.permute(0, 2, 1, 3).contiguous()
        attn_output = attn_output.reshape(batch_size, seq_len_per_rank, hidden_size)
        output = self.out_proj(attn_output)
        
        gathered_outputs = [torch.zeros_like(output) for _ in range(self.sp_size)]
        dist.all_gather(gathered_outputs, output)
        final_output = torch.cat(gathered_outputs, dim=1)
        
        return final_output

# Load model
if rank == 0:
    print(f"Loading Qwen 2.5 3B with 4D parallelism: DP={dp_size}, PP={pp_size}, TP={tp_size}, SP={sp_size}")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    torch_dtype=torch.float16,
    device_map=None
)

# Note: Full sequence parallelism integration would require modifying model architecture
# This is a conceptual demonstration

deepspeed_config = {
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": 1,
    "gradient_accumulation_steps": 1,
    "fp16": {"enabled": True},
    "zero_optimization": {"stage": 0},
    "pipeline": {"stages": pp_size},
    "tensor_parallel": {"tp_size": tp_size},
    "wall_clock_breakdown": True
}

model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    model_parameters=model.parameters(),
    config=deepspeed_config
)

# Data setup - MetaMathQA using shared utility
tokenized_dataset, tokenizer = load_metamathqa_dataset(
    split="train[:1000]",
    cache_dir="./metamathqa_tokenized_data",
    rank=rank
)

if rank == 0:
    print(f"MetaMathQA dataset loaded: {len(tokenized_dataset)} examples")

MICRO_BATCH_SIZE = 1
train_sampler = DistributedSampler(tokenized_dataset, num_replicas=world_size, rank=rank, shuffle=True)
train_loader = DataLoader(
    tokenized_dataset,
    batch_size=MICRO_BATCH_SIZE,
    sampler=train_sampler,
    collate_fn=lambda x: {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in x]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in x]),
        'labels': torch.stack([torch.tensor(item['labels']) for item in x])
    }
)

# Profiler trace handler
def trace_handler(prof):
    if rank == 0:
        trace_dir = "./profiler_logs/llm_4d_parallelism_trace"
        os.makedirs(trace_dir, exist_ok=True)
        prof.export_chrome_trace(f"{trace_dir}/rank{rank}_trace.json")
        print(f"Profiler trace saved to {trace_dir}/rank{rank}_trace.json")

# Training loop
model_engine.train()

# Profiler schedule: wait=1, warmup=1, active=3 (capture 3 steps), repeat=1
with torch.profiler.profile(
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
    on_trace_ready=trace_handler,
    record_shapes=True,
    with_stack=True,
    profile_memory=True
) as prof:
    
    for epoch in range(1):
        train_sampler.set_epoch(epoch)
        
        for i, batch in enumerate(train_loader):
            if i >= 20:
                break
            
            step_start_time = time.time()
            
            # Label training step
            with torch.profiler.record_function(f"## Training Step {i} ##"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                labels[labels == tokenizer.pad_token_id] = -100
                
                # Label forward pass
                with torch.profiler.record_function("## Forward Pass ##"):
                    outputs = model_engine(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                
                # Label backward pass
                with torch.profiler.record_function("## Backward Pass ##"):
                    model_engine.backward(loss)
                
                # Label optimizer step
                with torch.profiler.record_function("## Optimizer Step ##"):
                    model_engine.step()
            
            step_time = time.time() - step_start_time
            
            # Step profiler (only for first few steps)
            if i < 5:  # Profile first 5 steps (wait=1, warmup=1, active=3)
                prof.step()
        
        if rank == 0:
            mem_allocated = torch.cuda.memory_allocated() / 1e9
            effective_batch = MICRO_BATCH_SIZE * dp_size
            seq_len = input_ids.size(1)
            throughput_samples = effective_batch / step_time
            throughput_tokens = effective_batch * seq_len / step_time
            
            wandb.log({
                "loss": loss.item(),
                "step_time_sec": step_time,
                "memory_allocated_gb": mem_allocated,
                "throughput_samples_per_sec": throughput_samples,
                "throughput_tokens_per_sec": throughput_tokens
            })
            
            print(f"Step {i}: Loss={loss.item():.4f} | Time={step_time:.2f}s | Mem={mem_allocated:.2f}GB")

if rank == 0:
    wandb.finish()
dist.destroy_process_group()


In [ ]:
# Run 4D Parallelism experiment on 2 GPUs
# !deepspeed --num_gpus=2 train_4d_parallelism_qwen3b_metamath.py


# Part 5: 5D Parallelism - Adding Expert Parallelism (MoE)
## Full 5D Parallelism: Data + Pipeline + Tensor + Sequence + Expert Parallelism
## For Mixture-of-Experts Models


In [ ]:
%%writefile train_5d_parallelism_qwen3b_metamath.py
"""
5D Parallelism: Data + Pipeline + Tensor + Sequence + Expert Parallelism
Full 5D parallelism for Mixture-of-Experts (MoE) models with MetaMathQA dataset.
This demonstrates the complete parallelism framework.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import deepspeed
import torch.distributed as dist
from transformers import AutoModelForCausalLM
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader
from metamathqa_utils import load_metamathqa_dataset
import warnings
import wandb
import os
import time
import torch.profiler

warnings.filterwarnings("ignore")

# Setup
local_rank = int(os.environ['LOCAL_RANK'])
rank = int(os.environ['RANK'])
world_size = int(os.environ['WORLD_SIZE'])

torch.cuda.set_device(local_rank)
device = torch.device(f"cuda:{local_rank}")

# 5D Parallelism Configuration
if world_size == 2:
    dp_size = 1
    pp_size = 2
    tp_size = 1
    sp_size = 1
    ep_size = 1
elif world_size == 8:
    dp_size = 1
    pp_size = 2
    tp_size = 1
    sp_size = 1
    ep_size = 4
else:
    dp_size = 1
    pp_size = 2
    tp_size = 1
    sp_size = 1
    ep_size = world_size // (dp_size * pp_size * tp_size * sp_size)

deepspeed.init_distributed()

if rank == 0:
    wandb.init(
        project="LLM-Model-Parallelism-Qwen3B",
        name=f"5D-Parallelism-Qwen3B-MetaMathQA-{world_size}GPUs",
        config={
            "model": "Qwen2.5-3B",
            "dataset": "MetaMathQA",
            "parallelism": "5D-Full-Parallelism",
            "framework": "DeepSpeed + Custom MoE",
            "num_gpus": world_size,
            "data_parallel_size": dp_size,
            "pipeline_stages": pp_size,
            "tensor_parallel_size": tp_size,
            "sequence_parallel_size": sp_size,
            "expert_parallel_size": ep_size
        }
    )

# Mixture-of-Experts Layer with Expert Parallelism
class MoELayer(nn.Module):
    """
    Mixture-of-Experts layer demonstrating expert parallelism.
    Experts are distributed across GPUs.
    """
    def __init__(self, hidden_size, num_experts=4, expert_capacity=2, rank=0, world_size=1, ep_size=1):
        super().__init__()
        self.num_experts = num_experts
        self.expert_capacity = expert_capacity
        self.rank = rank
        self.world_size = world_size
        self.ep_size = ep_size
        
        experts_per_gpu = num_experts // ep_size
        expert_start = (rank % ep_size) * experts_per_gpu
        expert_end = expert_start + experts_per_gpu
        
        self.local_experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_size, hidden_size * 2),
                nn.GELU(),
                nn.Linear(hidden_size * 2, hidden_size)
            ) for _ in range(experts_per_gpu)
        ])
        
        self.local_expert_indices = list(range(expert_start, expert_end))
        self.gate = nn.Linear(hidden_size, num_experts)
        
    def forward(self, x):
        batch_size, seq_len, hidden_size = x.shape
        
        gate_logits = self.gate(x)
        gate_probs = F.softmax(gate_logits, dim=-1)
        
        top_k = 2
        top_k_gate_probs, top_k_indices = torch.topk(gate_probs, k=top_k, dim=-1)
        top_k_gate_probs = top_k_gate_probs / top_k_gate_probs.sum(dim=-1, keepdim=True)
        
        output = torch.zeros_like(x)
        
        for local_idx, global_expert_idx in enumerate(self.local_expert_indices):
            expert_mask = (top_k_indices == global_expert_idx).any(dim=-1)
            
            if expert_mask.any():
                expert_input = x[expert_mask]
                expert_probs = top_k_gate_probs[expert_mask]
                expert_positions = (top_k_indices[expert_mask] == global_expert_idx).nonzero(as_tuple=True)[1]
                
                expert_output = self.local_experts[local_idx](expert_input)
                expert_weights = expert_probs.gather(1, expert_positions.unsqueeze(1)).squeeze(1)
                expert_output = expert_output * expert_weights.unsqueeze(-1)
                
                output[expert_mask] += expert_output
        
        dist.all_reduce(output, op=dist.ReduceOp.SUM)
        return output

# Load base model
if rank == 0:
    print(f"Loading Qwen 2.5 3B with 5D parallelism:")
    print(f"  DP={dp_size}, PP={pp_size}, TP={tp_size}, SP={sp_size}, EP={ep_size}")

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    torch_dtype=torch.float16,
    device_map=None
)

# Note: Full MoE integration would require replacing MLP layers with MoE layers
# This is a conceptual demonstration of the architecture

deepspeed_config = {
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": 1,
    "gradient_accumulation_steps": 1,
    "fp16": {"enabled": True},
    "zero_optimization": {"stage": 0},
    "pipeline": {"stages": pp_size},
    "tensor_parallel": {"tp_size": tp_size},
    "wall_clock_breakdown": True
}

model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    model_parameters=model.parameters(),
    config=deepspeed_config
)

# Data setup - MetaMathQA using shared utility
tokenized_dataset, tokenizer = load_metamathqa_dataset(
    split="train[:1000]",
    cache_dir="./metamathqa_tokenized_data",
    rank=rank
)

if rank == 0:
    print(f"MetaMathQA dataset loaded: {len(tokenized_dataset)} examples")

MICRO_BATCH_SIZE = 1
train_sampler = DistributedSampler(tokenized_dataset, num_replicas=world_size, rank=rank, shuffle=True)
train_loader = DataLoader(
    tokenized_dataset,
    batch_size=MICRO_BATCH_SIZE,
    sampler=train_sampler,
    collate_fn=lambda x: {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in x]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in x]),
        'labels': torch.stack([torch.tensor(item['labels']) for item in x])
    }
)

# Profiler trace handler
def trace_handler(prof):
    if rank == 0:
        trace_dir = "./profiler_logs/llm_5d_parallelism_trace"
        os.makedirs(trace_dir, exist_ok=True)
        prof.export_chrome_trace(f"{trace_dir}/rank{rank}_trace.json")
        print(f"Profiler trace saved to {trace_dir}/rank{rank}_trace.json")

# Training loop
model_engine.train()

# Profiler schedule: wait=1, warmup=1, active=3 (capture 3 steps), repeat=1
with torch.profiler.profile(
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
    on_trace_ready=trace_handler,
    record_shapes=True,
    with_stack=True,
    profile_memory=True
) as prof:
    
    for epoch in range(1):
        train_sampler.set_epoch(epoch)
        
        for i, batch in enumerate(train_loader):
            if i >= 20:
                break
            
            step_start_time = time.time()
            
            # Label training step
            with torch.profiler.record_function(f"## Training Step {i} ##"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                labels[labels == tokenizer.pad_token_id] = -100
                
                # Label forward pass
                with torch.profiler.record_function("## Forward Pass ##"):
                    outputs = model_engine(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                
                # Label backward pass
                with torch.profiler.record_function("## Backward Pass ##"):
                    model_engine.backward(loss)
                
                # Label optimizer step
                with torch.profiler.record_function("## Optimizer Step ##"):
                    model_engine.step()
            
            step_time = time.time() - step_start_time
            
            # Step profiler (only for first few steps)
            if i < 5:  # Profile first 5 steps (wait=1, warmup=1, active=3)
                prof.step()
        
        if rank == 0:
            mem_allocated = torch.cuda.memory_allocated() / 1e9
            effective_batch = MICRO_BATCH_SIZE * dp_size
            seq_len = input_ids.size(1)
            throughput_samples = effective_batch / step_time
            throughput_tokens = effective_batch * seq_len / step_time
            
            wandb.log({
                "loss": loss.item(),
                "step_time_sec": step_time,
                "memory_allocated_gb": mem_allocated,
                "throughput_samples_per_sec": throughput_samples,
                "throughput_tokens_per_sec": throughput_tokens
            })
            
            print(f"Step {i}: Loss={loss.item():.4f} | Time={step_time:.2f}s | Mem={mem_allocated:.2f}GB")

if rank == 0:
    wandb.finish()
dist.destroy_process_group()


In [ ]:
# Run 5D Parallelism experiment on 2 GPUs
# !deepspeed --num_gpus=2 train_5d_parallelism_qwen3b_metamath.py


## Running Experiments

### Test on 2 GPUs First


In [ ]:
# Run all experiments on 2 GPUs with MetaMathQA
# Uncomment to run:

# 1D Parallelism (DDP)
# !torchrun --nproc_per_node=2 train_ddp_qwen3b_metamath.py

# 2D Parallelism (Pipeline + Data)
# !deepspeed --num_gpus=2 train_2d_pipeline_qwen3b_metamath.py

# 3D Parallelism (Data + Pipeline + Tensor)
# !deepspeed --num_gpus=2 train_3d_parallelism_qwen3b_metamath.py

# 4D Parallelism (Data + Pipeline + Tensor + Sequence)
# !deepspeed --num_gpus=2 train_4d_parallelism_qwen3b_metamath.py

# 5D Parallelism (Full 5D with MoE)
# !deepspeed --num_gpus=2 train_5d_parallelism_qwen3b_metamath.py


### Scale to 8 GPUs


In [ ]:
# Run all experiments on 8 GPUs with MetaMathQA
# Uncomment when you have 8 GPU access:

# 1D Parallelism (DDP)
# !torchrun --nproc_per_node=8 train_ddp_qwen3b_metamath.py

# 2D Parallelism (Pipeline + Data)
# !deepspeed --num_gpus=8 train_2d_pipeline_qwen3b_metamath.py

# 3D Parallelism (Data + Pipeline + Tensor)
# !deepspeed --num_gpus=8 train_3d_parallelism_qwen3b_metamath.py

# 4D Parallelism (Data + Pipeline + Tensor + Sequence)
# !deepspeed --num_gpus=8 train_4d_parallelism_qwen3b_metamath.py

# 5D Parallelism (Full 5D with MoE)
# !deepspeed --num_gpus=8 train_5d_parallelism_qwen3b_metamath.py


## Profiler Trace Analysis

### Profiling Setup

Each training script includes PyTorch Profiler that:
1. **Profiles first 3 training steps** (after 1 warmup step)
   - Schedule: `wait=1, warmup=1, active=3, repeat=1`
   - Captures detailed traces with memory profiling
2. **Saves Chrome traces** to `./profiler_logs/llm_*_trace/` directories
3. **Continues full training** after profiling completes

**Trace files saved:**
- `./profiler_logs/llm_1d_ddp_trace/rank{rank}_trace.json`
- `./profiler_logs/llm_2d_pipeline_trace/rank{rank}_trace.json`
- `./profiler_logs/llm_3d_parallelism_trace/rank{rank}_trace.json`
- `./profiler_logs/llm_4d_parallelism_trace/rank{rank}_trace.json`
- `./profiler_logs/llm_5d_parallelism_trace/rank{rank}_trace.json`

### Analyze Traces with HTA (Holistic Trace Analysis)


In [ ]:
# Install HTA for trace analysis
# !pip install hta

# After installing HTA, use this to analyze traces:
"""
from hta.trace_analysis import TraceAnalysis
import os

# Analyze traces for each parallelism strategy
trace_dirs = {
    "1D DDP": "./profiler_logs/llm_1d_ddp_trace",
    "2D Pipeline": "./profiler_logs/llm_2d_pipeline_trace",
    "3D Parallelism": "./profiler_logs/llm_3d_parallelism_trace",
    "4D Parallelism": "./profiler_logs/llm_4d_parallelism_trace",
    "5D Parallelism": "./profiler_logs/llm_5d_parallelism_trace"
}

for name, trace_dir in trace_dirs.items():
    if os.path.exists(trace_dir):
        print(f"\n{'='*20} Analyzing {name} {'='*20}")
        try:
            analyzer = TraceAnalysis(trace_dir=trace_dir)
            
            # Temporal breakdown
            temp_df = analyzer.get_temporal_breakdown(visualize=False)
            print(f"\nTemporal Breakdown:\n{temp_df}")
            
            # Communication vs Computation overlap
            overlap_df = analyzer.get_comm_comp_overlap(visualize=False)
            print(f"\nComm/Comp Overlap:\n{overlap_df}")
            
        except Exception as e:
            print(f"Analysis failed: {e}")
"""

print("HTA analysis code ready. Uncomment and run after collecting traces.")


## Results Analysis and Comparison

### Compare 2 GPU vs 8 GPU Performance


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# This cell will analyze results from WandB
# After running experiments, fetch results and compare

def analyze_scaling_efficiency(results_2gpu, results_8gpu):
    """
    Analyze scaling efficiency from 2 GPUs to 8 GPUs.
    """
    analysis = {}
    
    for parallelism_type in ['1D', '2D', '3D', '4D', '5D']:
        if parallelism_type in results_2gpu and parallelism_type in results_8gpu:
            throughput_2gpu = results_2gpu[parallelism_type]['throughput']
            throughput_8gpu = results_8gpu[parallelism_type]['throughput']
            
            ideal_speedup = 4.0  # 8/2
            actual_speedup = throughput_8gpu / throughput_2gpu
            efficiency = (actual_speedup / ideal_speedup) * 100
            
            analysis[parallelism_type] = {
                'throughput_2gpu': throughput_2gpu,
                'throughput_8gpu': throughput_8gpu,
                'actual_speedup': actual_speedup,
                'ideal_speedup': ideal_speedup,
                'efficiency_percent': efficiency
            }
    
    return analysis

# Example usage (after fetching from WandB):
# results_2gpu = {'1D': {'throughput': 10}, '2D': {'throughput': 12}, ...}
# results_8gpu = {'1D': {'throughput': 35}, '2D': {'throughput': 40}, ...}
# scaling_analysis = analyze_scaling_efficiency(results_2gpu, results_8gpu)

print("Analysis functions ready. Use after running experiments.")


## Summary

This notebook demonstrates progressive model parallelism strategies with **MetaMathQA** dataset:

1. **1D Parallelism**: Data Parallelism (DDP) - Baseline
2. **2D Parallelism**: Data + Pipeline Parallelism
3. **3D Parallelism**: Data + Pipeline + Tensor Parallelism
4. **4D Parallelism**: Data + Pipeline + Tensor + Sequence Parallelism
5. **5D Parallelism**: Full 5D with Expert Parallelism (MoE)

**Dataset**: MetaMathQA (395K math QA examples) - Industry-standard math reasoning dataset used by MetaMath-Mistral-7B, OpenChat-3.5, CausalLM, and other industry models.

**Model**: Qwen 2.5 3B (~3B parameters)

All experiments include comprehensive metrics:
- Model FLOPs Utilization (MFU)
- Memory usage (allocated, reserved, peak)
- Throughput (samples/sec, tokens/sec)
- Step time breakdown (forward, backward, optimizer)
- Communication overhead
- Scaling efficiency (2 GPU → 8 GPU)

**Next Steps**: Run experiments on 2 GPUs first, then scale to 8 GPUs for full analysis.


## Results Analysis and Comparison

### Compare 2 GPU vs 8 GPU Performance


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# This cell will analyze results from WandB
# After running experiments, fetch results and compare

def analyze_scaling_efficiency(results_2gpu, results_8gpu):
    """
    Analyze scaling efficiency from 2 GPUs to 8 GPUs.
    """
    analysis = {}
    
    for parallelism_type in ['1D', '2D', '3D', '4D', '5D']:
        if parallelism_type in results_2gpu and parallelism_type in results_8gpu:
            throughput_2gpu = results_2gpu[parallelism_type]['throughput']
            throughput_8gpu = results_8gpu[parallelism_type]['throughput']
            
            ideal_speedup = 4.0  # 8/2
            actual_speedup = throughput_8gpu / throughput_2gpu
            efficiency = (actual_speedup / ideal_speedup) * 100
            
            analysis[parallelism_type] = {
                'throughput_2gpu': throughput_2gpu,
                'throughput_8gpu': throughput_8gpu,
                'actual_speedup': actual_speedup,
                'ideal_speedup': ideal_speedup,
                'efficiency_percent': efficiency
            }
    
    return analysis

# Example usage (after fetching from WandB):
# results_2gpu = {'1D': {'throughput': 10}, '2D': {'throughput': 12}, ...}
# results_8gpu = {'1D': {'throughput': 35}, '2D': {'throughput': 40}, ...}
# scaling_analysis = analyze_scaling_efficiency(results_2gpu, results_8gpu)

print("Analysis functions ready. Use after running experiments.")
